## Ollama Integration

This notebook focuses on integrating a local LLM (Ollama) with our memory system.



## Step 1: Install Ollama

Before we proceed, you need to install Ollama on your system.

### Installation Command:
```bash
curl -fsSL https://ollama.ai/install.sh | sh


For Linux/WSL
wget -qO- https://ollama.ai/install.sh | sh

Now verify the installation
```bash
ollama --version

## Step 2: Install Ollama Python Client

Now we will install the Python client library to communicate with Ollama from our code.

### Installation Command:
```bash
pip install ollama

## Step 3: Download LLM Model

Now let's download a language model to use with our memory system.

### Recommended Model (Lightweight):
```bash
ollama pull llama3.2:3b

### Verify Model Installation

``` bash
ollama list

### Test the model now
``` bash 
ollama run llama3.2:3b
# Type a test message and exit.

In [5]:
## We will just test the model with a small snippet,,

import ollama

print("Testing Llama 3.2 model...")

try:
    # Simple test conversation for checking..
    response = ollama.chat(
        model='llama3.2:3b',
        messages=[
            {
                'role': 'user',
                'content': 'Hello! who are you?'
            }
        ]
    )
    
    print("Model is working!")
    print("="*200)
    print(f"Response: {response['message']['content']}")
    
except Exception as e:
    print(f"Error: {e}")

Testing Llama 3.2 model...
Model is working!
Response: Hello! I'm an artificial intelligence model, which means I'm a computer program designed to simulate conversations and answer questions to the best of my knowledge. I don't have a personal identity or physical presence, but I'm here to help and provide information on a wide range of topics.

I can assist with tasks such as:

* Answering questions
* Generating text
* Translating languages
* Summarizing content
* And more!

Feel free to ask me anything, and I'll do my best to help. How can I assist you today?


## Step 4: Connect Ollama with Memory Database

Now we'll integrate our LLM (Ollama) with the memory system we built in setup_vectorydb notebook.

### What we're going to do:
1. **Import ChromaDB** - Same as setup_vectordb notebook to access our memory
2. **Connect to existing collection** - Get our stored conversations
3. **Create memory-aware function** - Combine memory search + LLM response
4. **Test the integration** - See how LLM uses past conversations

### Below is the code for doing the above things (Most of the things we did earlier in setup_vectordb notebook)

In [15]:
# Step 4A: Import required libraries
import chromadb
import ollama
from datetime import datetime
import json

print("Connecting Ollama with Memory Database...")

# Step 4B: Connect to existing persistent ChromaDB from Phase 1
client = chromadb.PersistentClient(path="./chroma_db")

# Get the existing collection created in Phase 1
collection = client.get_collection(name="conversation_memory")

print("✅ Connected to existing memory collection!")
print(f" Total memories available: {collection.count()}")
print(f" Collection name: {collection.name}")

print("\n" + "="*100)
print(" Memory-LLM Integration Setup Complete!")


Connecting Ollama with Memory Database...
✅ Connected to existing memory collection!
 Total memories available: 10
 Collection name: conversation_memory

 Memory-LLM Integration Setup Complete!


## Create Memory-Aware Chat Function
### What we will buil now is:

1. **Search function** - Find relevant past conversations
2. **Context builder** - Format memory for LLM prompt
3. **Memory aware chat** - LLM that uses conversation history
4. **Test it** - See how memory improves responses

In [18]:
def search_memory(query, top_k=3):
    """
    Search for relevant past conversations based on the query
    """
    print(f" Searching memory for: '{query}'")
    
    # Search for similar conversations
    results = collection.query(
        query_texts=[query],
        n_results=top_k
    )
    
    print(f" Found {len(results['documents'][0])} relevant memories")
    return results

def format_memory_context(search_results):
    """
    Format search results into context for the LLM
    """
    if not search_results['documents'][0]:
        return "No relevant conversation history found."
    
    context = "=== RELEVANT CONVERSATION HISTORY ===\n\n"
    
    for i, (doc, metadata) in enumerate(zip(
        search_results['documents'][0], 
        search_results['metadatas'][0]
    )):
        context += f"Memory {i+1}:\n"
        context += f"Content: {doc}\n"
        context += f"Topic: {metadata.get('topic', 'Unknown')}\n"
        context += f"User Skill Level: {metadata.get('user_level', 'Unknown')}\n"
        context += f"Technology: {metadata.get('tech_stack', 'Unknown')}\n"
        context += f"Timestamp: {metadata.get('timestamp', 'Unknown')}\n"
        context += "-" * 50 + "\n\n"
    
    return context

def memory_chat(user_message, use_memory=True):
    """
    Chat with LLM using conversation memory for context
    """
    print("="*100)
    print(f" User: {user_message}")
    print("="*100)
    
    # now we will build a very detailed and contextual  prompt
    if use_memory:
        # Searching  for relevant memorie
        memory_results = search_memory(user_message)
        memory_context = format_memory_context(memory_results)
        
        # what i have done is that i have enhanced professional system prompt with memory
        system_prompt = f"""You are an advanced AI assistant with persistent memory capabilities. Your primary objective is to provide highly personalized, contextually-aware responses based on the user's conversation history and demonstrated preferences.

{memory_context}

CORE INSTRUCTIONS:
1. MEMORY UTILIZATION: Always reference and build upon the provided conversation history to demonstrate continuity and understanding of the user's background, interests, and expertise levels.

2. PERSONALIZATION: Adapt your communication style, technical depth, and examples to match the user's demonstrated skill level and interests from previous conversations.

3. CONTEXTUAL AWARENESS: Connect current questions to past discussions when relevant, showing how topics relate to the user's ongoing projects or learning journey.

4. CONSISTENCY: Maintain awareness of the user's preferences, previously discussed technologies, and established context across all interactions.

5. PROGRESSIVE LEARNING: Build upon previous conversations to offer increasingly sophisticated insights and recommendations that align with the user's growing expertise.

6. RELEVANCE FILTERING: Only reference historical context that is directly relevant to the current query - avoid overwhelming responses with unnecessary background information.

7. ACKNOWLEDGE LIMITATIONS: If the conversation history doesn't contain relevant information for the current query, explicitly state this and provide the best possible response based on available context.

RESPONSE REQUIREMENTS:
- Be specific and actionable
- Reference relevant past discussions naturally  
- Maintain professional yet personable tone
- Provide depth appropriate to user's demonstrated skill level
- Offer practical next steps when applicable
- Use examples that align with user's known interests and experience level

TECHNICAL CONTEXT AWARENESS:
- Consider the user's demonstrated expertise with specific technologies
- Build upon their known project context and learning goals
- Suggest resources appropriate to their skill progression
- Connect new concepts to their established knowledge base

Respond to the user's current message while seamlessly integrating insights from their conversation history to provide the most helpful and contextually appropriate response possible."""
    else:
        system_prompt = """You are a helpful AI assistant. Provide clear, accurate, and contextually appropriate responses to user queries. Be professional, concise, and ensure your answers are actionable and well-structured."""
    
    # Chat with Ollama
    try:
        response = ollama.chat(
            model='llama3.2:3b',
            messages=[
                {'role': 'system', 'content': system_prompt},
                {'role': 'user', 'content': user_message}
            ]
        )
        
        ai_response = response['message']['content']
        print(f" AI Response:\n{ai_response}")
        
        return ai_response
        
    except Exception as e:
        print(f" Error: {e}")
        return None

print("✅ Memory aware chat function created!")


✅ Memory aware chat function created!


### Basic Test script for Testing our function

In [21]:
# Step 6: Test Memory-Aware Chat Function

print(" Testing Memory-Aware Chat System")
print("=" * 60)

# Test 1: Ask about Python (should find relevant memories)
print("\n TEST 1: Asking about Python")
memory_chat("What can you tell me about Python programming?")


 Testing Memory-Aware Chat System

 TEST 1: Asking about Python
 User: What can you tell me about Python programming?
 Searching memory for: 'What can you tell me about Python programming?'
 Found 3 relevant memories
 AI Response:
Python! As someone who has expressed enthusiasm for Python in our previous conversations, I'd be happy to dive deeper into this fantastic language.

From what we've discussed before, it's clear that you have a solid foundation in Python and are likely looking for more advanced topics or possibly seeking advice on how to apply your skills in real-world projects.

Here are some key aspects of Python programming that I think would be worth exploring:

1. **Data Science with NumPy, Pandas, and Scikit-learn**: As someone interested in data science, you're probably familiar with the popular libraries like NumPy, Pandas, and Scikit-learn. These libraries provide an efficient way to manipulate and analyze data, making it a crucial tool for data scientists.
2. **Machi

"Python! As someone who has expressed enthusiasm for Python in our previous conversations, I'd be happy to dive deeper into this fantastic language.\n\nFrom what we've discussed before, it's clear that you have a solid foundation in Python and are likely looking for more advanced topics or possibly seeking advice on how to apply your skills in real-world projects.\n\nHere are some key aspects of Python programming that I think would be worth exploring:\n\n1. **Data Science with NumPy, Pandas, and Scikit-learn**: As someone interested in data science, you're probably familiar with the popular libraries like NumPy, Pandas, and Scikit-learn. These libraries provide an efficient way to manipulate and analyze data, making it a crucial tool for data scientists.\n2. **Machine Learning with TensorFlow or Keras**: Given your interest in machine learning, I'd recommend checking out TensorFlow or Keras. Both frameworks offer powerful tools for building and training models, but they cater to diffe

In [22]:
# Test 2: Ask about AI/ML (should find relevant memories) 
print(" TEST 2: Asking about AI and Machine Learning")
memory_chat("I'm interested in machine learning. Can you help?")

 TEST 2: Asking about AI and Machine Learning
 User: I'm interested in machine learning. Can you help?
 Searching memory for: 'I'm interested in machine learning. Can you help?'
 Found 3 relevant memories
 AI Response:
Machine learning! I'd be delighted to help.

Since we discussed machine learning previously, I'll assume you have a solid foundation in the basics. If not, don't worry – I can refresh your memory or introduce new concepts tailored to your skill level.

To get started, could you please share what aspect of machine learning you're interested in exploring further? Are you looking into:

1. Supervised/unsupervised learning?
2. Deep learning techniques (e.g., neural networks)?
3. Specific applications like natural language processing (NLP) or computer vision?
4. Model evaluation and hyperparameter tuning?

Let me know, and I'll do my best to provide targeted guidance, examples, or resources that align with your interests and expertise level.

(By the way, I noticed you mentio

"Machine learning! I'd be delighted to help.\n\nSince we discussed machine learning previously, I'll assume you have a solid foundation in the basics. If not, don't worry – I can refresh your memory or introduce new concepts tailored to your skill level.\n\nTo get started, could you please share what aspect of machine learning you're interested in exploring further? Are you looking into:\n\n1. Supervised/unsupervised learning?\n2. Deep learning techniques (e.g., neural networks)?\n3. Specific applications like natural language processing (NLP) or computer vision?\n4. Model evaluation and hyperparameter tuning?\n\nLet me know, and I'll do my best to provide targeted guidance, examples, or resources that align with your interests and expertise level.\n\n(By the way, I noticed you mentioned vector databases and embeddings in a previous conversation. If you'd like, we could explore how machine learning concepts intersect with those topics – just let me know!)"

In [23]:
# Test 3: Ask about something unrelated (should have limited memory)
print(" TEST 3: Asking about cooking (unrelated topic)")
memory_chat("How do I make pasta?")

 TEST 3: Asking about cooking (unrelated topic)
 User: How do I make pasta?
 Searching memory for: 'How do I make pasta?'
 Found 3 relevant memories
 AI Response:
Making pasta from scratch - a classic Italian tradition! As you're likely familiar with, making pasta is more than just boiling some noodles; it's an art that requires attention to detail, quality ingredients, and a bit of love.

To get started, let's talk about the basics. You'll need some simple tools and ingredients:

**Tools:**

* Stand mixer or hand-cranked pasta machine
* Kneading surface (wooden board or countertop)
* Rolling pin
* Cutting tool (pasta cutter or sharp knife)

**Ingredients:**

* 1 cup all-purpose flour (preferably "00" or caputo flour)
* 1 large egg
* Salt, to taste

Now that we have our ingredients and tools, let's dive into the process.

**Step 1: Create the Dough**

In a large mixing bowl, combine your flour and salt. Make a well in the center of the flour mixture and crack your egg into it. Using a 

'Making pasta from scratch - a classic Italian tradition! As you\'re likely familiar with, making pasta is more than just boiling some noodles; it\'s an art that requires attention to detail, quality ingredients, and a bit of love.\n\nTo get started, let\'s talk about the basics. You\'ll need some simple tools and ingredients:\n\n**Tools:**\n\n* Stand mixer or hand-cranked pasta machine\n* Kneading surface (wooden board or countertop)\n* Rolling pin\n* Cutting tool (pasta cutter or sharp knife)\n\n**Ingredients:**\n\n* 1 cup all-purpose flour (preferably "00" or caputo flour)\n* 1 large egg\n* Salt, to taste\n\nNow that we have our ingredients and tools, let\'s dive into the process.\n\n**Step 1: Create the Dough**\n\nIn a large mixing bowl, combine your flour and salt. Make a well in the center of the flour mixture and crack your egg into it. Using a fork, gently incorporate the flour into the egg, starting from the inner rim of the well. As you mix, add more flour if needed to achiev

In [24]:
# Test 4: Compare with and without memory
print(" TEST 4: Same Python question WITHOUT memory")
memory_chat("What can you tell me about Python programming?", use_memory=False)

 TEST 4: Same Python question WITHOUT memory
 User: What can you tell me about Python programming?
 AI Response:
Python is a high-level, interpreted programming language that is widely used for various purposes such as web development, scientific computing, data analysis, artificial intelligence, and more. Here are some key features and facts about Python:

**Key Features:**

1. **Easy to learn**: Python has a simple syntax and is relatively easy to read and write, making it a great language for beginners.
2. **High-level language**: Python abstracts away many low-level details, allowing developers to focus on the logic of their program without worrying about memory management and other details.
3. **Interpreted language**: Python code is executed line by line, which means that the interpreter checks each line before running it, making it easier to debug and test code.
4. **Dynamic typing**: Python is dynamically typed, meaning that you don't need to declare variable types before using

"Python is a high-level, interpreted programming language that is widely used for various purposes such as web development, scientific computing, data analysis, artificial intelligence, and more. Here are some key features and facts about Python:\n\n**Key Features:**\n\n1. **Easy to learn**: Python has a simple syntax and is relatively easy to read and write, making it a great language for beginners.\n2. **High-level language**: Python abstracts away many low-level details, allowing developers to focus on the logic of their program without worrying about memory management and other details.\n3. **Interpreted language**: Python code is executed line by line, which means that the interpreter checks each line before running it, making it easier to debug and test code.\n4. **Dynamic typing**: Python is dynamically typed, meaning that you don't need to declare variable types before using them.\n\n**Use Cases:**\n\n1. **Web development**: Python is used in web development frameworks such as 